In [1]:
import sys
sys.executable

'/media/rehan/yaws/ws_new/OpenKE/py/bin/python3.7'

In [2]:
import torch
import os
import sys
sys.path.append('../')
from models import *
import config
import numpy as np

In [12]:
from pymongo import MongoClient
db_name = 'freebase'
collection_name = 'fb15k237'
MONGO_DB_URL = "mongodb://localhost:27017/" # currently just using a local db
client = MongoClient(MONGO_DB_URL)
coll = client[db_name][collection_name]

In [70]:
in_path = '../benchmarks/tim_minikb/'
# in_path = '../benchmarks/FB15K237_names/'
with open( in_path + 'entity2id.txt') as nf:
    lines = [line.split('\t') for line in nf.readlines()[1:]]
    ent2id = {line[0]:int(line[1]) for line in lines}
ent2id   

{'Canada': 0,
 'Николай Яковына': 1,
 '0df25f25-888c-47a7-af11-5c8b0ac40a91': 2,
 'Валентина Ачкасова': 3,
 'Маркияин Лубкивский': 4,
 'Pueblo United': 5,
 '02ada19f-f28d-4ab4-a7c2-9d42bc51cb2d': 6,
 'edda5b09-d064-4224-903e-5e36318bb75f': 7,
 '8c789821-a465-497f-ac70-e2eb71d47333': 8,
 'a33e497b-094a-4598-8fe2-9dc476c88cd9': 9,
 'Paw Sox': 10,
 '10fa30b5-ce6d-483d-83e9-630a3edac16f': 11,
 '74b8192d-aa13-4433-81c7-ede2665ac5bc': 12,
 '41b67b62-8e0c-4713-afeb-d5d488186ac6': 13,
 '377c1bc6-42f2-445c-b163-227f9ccc73cb': 14,
 'f4dbbcb4-68b1-432b-85a3-659b8cca04f9': 15,
 'f9c41871-d53b-4b46-bc29-dc653a23cf3d': 16,
 "C's": 17,
 'you': 18,
 'f5dbf2e2-5f67-44c2-874b-fc3a538b047c': 19,
 'Путина': 20,
 '7aa7a49b-e46f-42dd-8dfc-84893c24f394': 21,
 'company': 22,
 '56f66624-b38a-48b3-b330-28217c008e25': 23,
 '8ba82886-4f1b-4f16-b784-599140d853de': 24,
 "NBA's": 25,
 "Giron's": 26,
 'Gregor Blanco': 27,
 'a98bd5bb-2336-4725-b90e-38ee0edae9e4': 28,
 '86e29040-78d2-438d-9e08-e71f68dda7d0': 29,
 'him'

In [18]:
import numpy as np
import re

In [24]:
valid_id_names = [id for id in ent2id.keys() if re.search(r'\w+-\w+-\w+-.*', id)]
valid_ids = [ent2id[id] for id in valid_id_names]

In [ ]:
embedding()

In [13]:
name2ent = {}
for entry in coll.find({}, {'@id':1, 'name':1}):
    if 'name' in entry:
        name2ent[entry['@id']] = entry['name']

In [14]:
sorted_items = sorted(name2ent.items(), key=lambda x: x[1])

In [15]:
from collections import defaultdict
name2id = defaultdict(list)
for item in sorted_items:
    name2id[item[1]] += [item[0]]
    

In [37]:
coref_ents = [ents for ents in name2id.values() if len(ents) > 1]
coref_ids = [[ent2id[ent] for ent in ents] for ents in coref_ents]

In [179]:
name2id_partial = defaultdict(list)
name_keeper = defaultdict(list)
for item in sorted_items:
    words = item[1].split(' ')
    bigrams = ['_'.join([w1, w2]) for w1, w2 in zip(words[:-1], words[1:])]
    for big in bigrams:
        if item[1] not in name_keeper[big]:
            name2id_partial[big] += [item[0]]
            name_keeper[big] += [item[1]]
coref_ents_partial = [ents for ents in name2id_partial.values() if len(ents) > 5 and len(ents) < 10]
coref_ids_partial = [[ent2id[ent] for ent in ents if ent in ent2id] for ents in coref_ents_partial ]

In [180]:
len(coref_ids_partial)

83

In [5]:
from scipy import sparse
tf_idf_mat = sparse.load_npz(in_path + 'ent_tfidf.pkl.npz')
# count_mat = sparse.load_npz('../models/ent_tf.pkl.npz')

In [6]:
con2 = config.Config()
con2.set_in_path(in_path)
con2.init()

In [185]:
te = TransE(con2)
te.load_state_dict(torch.load('../result/TransE_no_names.ckpt'))

/media/rehan/yaws/ws_new/OpenKE/py/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
../models/TransE.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.ent_embeddings.weight.data)
../models/TransE.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(self.rel_embeddings.weight.data)


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [174]:
def sim_score_embedding(embedding, co_ids):
    avg_values = []
    avg_count = 0
    for coref_set in co_ids:
        count = 0
        value = 0.
        for i in range(len(coref_set)):
            for j in range(i+1, len(coref_set)):
                id_i = torch.tensor([coref_set[i]])
                id_j = torch.tensor([coref_set[j]])
                value += np.maximum(-1, torch.nn.functional.cosine_similarity(embedding(id_i), 
                                                                         embedding(id_j)).data.numpy())
                count += 1
        avg_values.append(value/count)
    return np.array(avg_values),np.mean(avg_values)   

In [186]:
te_sim_score = sim_score_embedding(te.ent_embeddings, coref_ids_partial)
print(te_sim_score)

(array([[0.81306773],
       [0.43258595],
       [0.71899116],
       [0.7672828 ],
       [0.76053077],
       [0.36215118],
       [0.514485  ],
       [0.45366934],
       [0.3603166 ],
       [0.40926203],
       [0.6414679 ],
       [0.8890412 ],
       [0.8890412 ],
       [0.74284875],
       [0.6517553 ],
       [0.30620295],
       [0.49366048],
       [0.36314985],
       [0.55032825],
       [0.5103051 ],
       [0.58395576],
       [0.24964884],
       [0.41699794],
       [0.3938449 ],
       [0.23906715],
       [0.47304597],
       [0.35163802],
       [0.40934116],
       [0.443966  ],
       [0.3637782 ],
       [0.58202505],
       [0.72149616],
       [0.5726113 ],
       [0.5726113 ],
       [0.5726113 ],
       [0.5726113 ],
       [0.52017635],
       [0.19704047],
       [0.4007012 ],
       [0.7218854 ],
       [0.68222916],
       [0.73552316],
       [0.23501305],
       [0.13308862],
       [0.6376876 ],
       [0.4107875 ],
       [0.7284104 ],
       [0.58

In [187]:
print(te_sim_score[0].min())
print(te_sim_score[0].max())

0.13308862
0.8890412


In [7]:
cte_idf_bias = CharTransE(con2)
cte_idf_bias.load_state_dict(torch.load('../result/CharTransE.ckpt'))

/media/rehan/yaws/ws_new/OpenKE/py/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [15]:
id2tens_idf[0]

tensor(indices=tensor([[   0,    0,    0,    0,    0],
                       [7904, 7902, 7668, 8252, 8281]]),
       values=tensor([0.4777, 0.4307, 0.4155, 0.4307, 0.4777]),
       size=(1, 8434), nnz=5, layout=torch.sparse_coo)

In [16]:
print(sim(cte_idf_bias.ent_char_weights, 0, 1))

tensor([-0.2962], grad_fn=<DivBackward0>)


In [32]:
def embedding(weights, tensor):
    return weights(tensor).data.numpy().reshape((-1,))

In [33]:
ent_embeddings = np.array([embedding(cte_idf_bias.ent_char_weights, id2tens_idf[id_]) for id_ in valid_ids])

In [42]:
ent_embeddings = ent_embeddings/np.linalg.norm(ent_embeddings, axis=1).reshape((-1,1))
sim_score = ent_embeddings.dot(ent_embeddings.T) - np.eye(ent_embeddings.shape[0])

In [54]:
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
mat = csr_matrix(sim_score > 0.8)


In [57]:
n_components, labels = connected_components(csgraph=mat, directed=False, return_labels=True)
from collections import defaultdict
comps = defaultdict(list)
for i, lab in enumerate(labels):
    comps[lab] += [i]

In [60]:
comps

defaultdict(list,
            {0: [0],
             1: [1],
             2: [2],
             3: [3],
             4: [4],
             5: [5],
             6: [6],
             7: [7],
             8: [8],
             9: [9],
             10: [10],
             11: [11],
             12: [12],
             13: [13],
             14: [14],
             15: [15],
             16: [16],
             17: [17],
             18: [18],
             19: [19, 102],
             20: [20],
             21: [21],
             22: [22],
             23: [23],
             24: [24],
             25: [25],
             26: [26],
             27: [27],
             28: [28],
             29: [29],
             30: [30],
             31: [31],
             32: [32],
             33: [33],
             34: [34],
             35: [35],
             36: [36],
             37: [37],
             38: [38],
             39: [39],
             40: [40],
             41: [41],
             42: [42],
        

In [64]:
comps = [comp for comp in comps.values() if len(comp) > 1]

AttributeError: 'list' object has no attribute 'values'

In [65]:
comps

[[19, 102],
 [43, 224, 395],
 [83, 210, 426],
 [104, 296],
 [130, 379],
 [134, 419],
 [142, 391],
 [291, 418],
 [342, 362]]

In [69]:
print(valid_id_names[342])
print(valid_id_names[362])
print(valid_id_names[395])

ab3d1c87-9976-4e63-a9f7-668881b33f6c
e68802f0-683b-4914-b921-d1d4ab93dca1
7346b61e-59ca-470f-bfb6-f017f553c72b


In [10]:
id2tens_idf = {
    i:sparse_tensor(cte_idf_bias.tf_idf_mat[i,:]) for i in range(cte_idf_bias.tf_idf_mat.shape[0])
}

In [69]:
id2tens_count = {
    i:sparse_tensor(count_mat[i,:]) for i in range(count_mat.shape[0])
}

In [13]:
def sim(weights, id1, id2):
    
    emb_i = weights(id2tens_idf[id1])
    emb_j = weights(id2tens_idf[id2])
    return torch.nn.functional.cosine_similarity(emb_i, emb_j)

def sim_score(weights, id2tens, co_ids):
    avg_values = []
    avg_count = 0
    for coref_set in co_ids:
        count = 0
        value = 0.
        for i in range(len(coref_set)):
            for j in range(i+1, len(coref_set)):
                
                x_i = id2tens[coref_set[i]].cuda()
                x_j = id2tens[coref_set[j]].cuda()
                
                emb_i = weights(x_i)
                emb_j = weights(x_j)
                
                value += np.maximum(-1, torch.nn.functional.cosine_similarity(emb_i, emb_j).cpu().data.numpy())
                count += 1
        avg_values.append(value/count)
    return np.array(avg_values), np.mean(avg_values)

In [183]:
sim_cte_bias = sim_score(cte_idf_bias.ent_char_weights, id2tens_idf, coref_ids_partial)
print(sim_cte_bias)

(array([[0.69832504],
       [0.58405584],
       [0.76596755],
       [0.6338045 ],
       [0.71206653],
       [0.5407865 ],
       [0.6148109 ],
       [0.71615857],
       [0.51310754],
       [0.4581888 ],
       [0.6661923 ],
       [0.8559558 ],
       [0.8559558 ],
       [0.65710014],
       [0.6376719 ],
       [0.58242947],
       [0.61622953],
       [0.5061602 ],
       [0.6351069 ],
       [0.6023047 ],
       [0.7159161 ],
       [0.5129976 ],
       [0.5796582 ],
       [0.5420595 ],
       [0.5492078 ],
       [0.62993586],
       [0.5286878 ],
       [0.6041726 ],
       [0.55083317],
       [0.6613682 ],
       [0.6677304 ],
       [0.7441263 ],
       [0.80623513],
       [0.80623513],
       [0.80623513],
       [0.80623513],
       [0.68779826],
       [0.5505182 ],
       [0.5673676 ],
       [0.77318776],
       [0.7148434 ],
       [0.72206116],
       [0.46948263],
       [0.35289806],
       [0.7021592 ],
       [0.55785173],
       [0.75882816],
       [0.63

In [184]:
print(sim_cte_bias[0].min())
print(sim_cte_bias[0].max())

0.35289806
0.9054452


In [115]:
char_weights_no_bias = torch.load('../result/CharTransE_idf_nb.ckpt')
char_weights_no_bias["ent_char_weights.bias"] = torch.zeros(100)
cte_idf_bias_no_bias = CharTransE(con)
cte_idf_bias_no_bias.load_state_dict(char_weights_no_bias)

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [188]:
sim_cte_no_bias = sim_score(cte_idf_bias_no_bias.ent_char_weights, id2tens_idf, coref_ids_partial)
print(sim_cte_no_bias)

(array([[0.7005533 ],
       [0.5834661 ],
       [0.77875894],
       [0.6587926 ],
       [0.7092466 ],
       [0.5029194 ],
       [0.5455037 ],
       [0.69477695],
       [0.5334169 ],
       [0.46115094],
       [0.6837609 ],
       [0.8354438 ],
       [0.8354438 ],
       [0.68102586],
       [0.65851766],
       [0.6002253 ],
       [0.6265582 ],
       [0.48609743],
       [0.6169167 ],
       [0.5680682 ],
       [0.724353  ],
       [0.54887027],
       [0.63859856],
       [0.565674  ],
       [0.50980234],
       [0.5941651 ],
       [0.50110763],
       [0.5826691 ],
       [0.54720217],
       [0.646114  ],
       [0.66787255],
       [0.7363353 ],
       [0.815467  ],
       [0.815467  ],
       [0.815467  ],
       [0.815467  ],
       [0.67679197],
       [0.525529  ],
       [0.54554176],
       [0.779484  ],
       [0.7065534 ],
       [0.772813  ],
       [0.43890566],
       [0.3223173 ],
       [0.6562246 ],
       [0.54882634],
       [0.770289  ],
       [0.62

In [118]:
cte_count = CharTransE(con)
cte_idf_bias_no_bias.load_state_dict(torch.load('../result/CharTransE_tf.ckpt'))

/media/rehan/yaws/ws_new/OpenKE/py/lib/python3.7/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [189]:
sim_cte_count = sim_score(cte_count.ent_char_weights, id2tens_count, coref_ids_partial)
print(sim_cte_count)

(array([[ 0.05259552],
       [ 0.12667416],
       [ 0.11905514],
       [ 0.02837177],
       [ 0.03535303],
       [ 0.04042268],
       [ 0.05110943],
       [ 0.07968297],
       [ 0.12653011],
       [ 0.01929603],
       [ 0.05746136],
       [ 0.01771273],
       [ 0.01771273],
       [ 0.03845565],
       [ 0.016543  ],
       [ 0.07846709],
       [ 0.02071244],
       [-0.00608002],
       [ 0.04395593],
       [ 0.04801859],
       [ 0.06866635],
       [ 0.04363069],
       [ 0.09744488],
       [ 0.10168979],
       [ 0.08685375],
       [ 0.04820091],
       [ 0.03734241],
       [ 0.04189683],
       [ 0.06484944],
       [ 0.00394111],
       [ 0.03455067],
       [ 0.08236219],
       [ 0.29542673],
       [ 0.29542673],
       [ 0.29542673],
       [ 0.29542673],
       [ 0.07125137],
       [ 0.04847551],
       [ 0.06800718],
       [ 0.10291163],
       [ 0.01491673],
       [ 0.07174192],
       [ 0.04752094],
       [ 0.08008324],
       [ 0.05135889],
       [ 

In [191]:
print(sim_cte_count[0].min())

-0.006080025


In [192]:
name2ent

{'kg:/m/029jpy': 'New England',
 'kg:/m/03ksy': 'Harvard University',
 'kg:/m/0d1_f': 'Elizabeth II',
 'kg:/m/0f2v0': 'Miami',
 'kg:/m/088q4': 'Zimbabwe',
 'kg:/m/0g_w': 'Academy Awards',
 'kg:/m/0d06m5': 'Hillary Clinton',
 'kg:/m/0tbql': 'Wichita',
 'kg:/m/01v0sx2': 'U.S.A. for Africa',
 'kg:/m/01w5m': 'Columbia University',
 'kg:/m/0f1_p': 'Gujarat',
 'kg:/m/04n3l': 'Long Island',
 'kg:/m/07bxhl': 'Bhutan',
 'kg:/m/01w92': 'Channel 4',
 'kg:/m/0212mp': 'Burnley F.C.',
 'kg:/m/0134s5': 'AC/DC',
 'kg:/m/05j49': 'Newfoundland and Labrador',
 'kg:/m/01d88c': 'Ahmedabad',
 'kg:/m/0pspl': 'Georgetown University',
 'kg:/m/0phrl': 'Days of Our Lives',
 'kg:/m/01vw26l': 'Ice Cube',
 'kg:/m/02183k': 'University of Georgia',
 'kg:/m/0cxn2': 'Ice cream',
 'kg:/m/0fqyc': 'North Holland',
 'kg:/m/0bq2g': 'Gwyneth Paltrow',
 'kg:/m/01vv6_6': 'Slash',
 'kg:/m/0gxkm': 'Parma Calcio 1913',
 'kg:/m/07cyl': 'The Silence of the Lambs',
 'kg:/m/016h4r': 'Kris Kristofferson',
 'kg:/m/03hnd': 'H. G. Wells'

In [193]:
name2ent['kg:/m/0xnvg']

KeyError: 'kg:/m/0xnvg'